<left>FINM 32000 - Numerical Methods</left>
<left>Spring 2023</left>
<br>
<h1><center> Homework 2 </center></h1>
<center>Due - 23:59 [CST] April 13th, 2023</center>
<br>
<h3>Ki Hyun</h3>
<h3>Student ID: 12125881</h3>

#### Imports

In [1]:
import numpy as np

#### Helper Functions

In [2]:
class localvolDynamics:
    
    def __init__(self, S0, r, q, maxvol, localvol): 
        self.S0 = S0
        self.r = r
        self.q = q
        self.maxvol = maxvol
        self.localvol = localvol


In [3]:
class CallOnAmericanPut:

    def __init__(self, putexpiry, putstrike, callexpiry, callstrike):
        self.putexpiry = putexpiry
        self.putstrike = putstrike
        self.callexpiry = callexpiry
        self.callstrike = callstrike


In [4]:
class TrinomTree:
    
    def __init__(self, N):
        self.N = N
        
    # You complete the coding of this function
    def price_compound_localvol(self, contract, dynamics):

        # initial local vol
        sigma_avg = dynamics.localvol(dynamics.S0, 0)

        # constants throughout the pricing
        deltat = contract.putexpiry / self.N
        rGrow = dynamics.r - dynamics.q
        disc = np.exp(-dynamics.r * deltat)
        deltax = np.maximum(sigma_avg * np.sqrt(3 * deltat), dynamics.maxvol * np.sqrt(deltat))

        # up, down, middle probabilities:
        def prob(S, t, direction):
            if direction == "u":
                sigma = dynamics.localvol(S, t)
                nu = rGrow - sigma**2 / 2
                P = ((sigma**2 * deltat + nu**2 * deltat**2) / (deltax**2) + nu * deltat / deltax)/2
            elif direction == "m":
                sigma = dynamics.localvol(S, t)
                nu = rGrow - sigma**2 / 2
                P = 1 - (sigma**2 * deltat + nu**2 * deltat**2) / (deltax**2)
            elif direction == "d":
                sigma = dynamics.localvol(S, t)
                nu = rGrow - sigma**2 / 2
                P = ((sigma**2 * deltat + nu**2 * deltat**2) / (deltax**2) - nu * deltat / deltax)/2
            else:
                raise Exception("Direction in a trinomial tree should be 'u', 'm', or 'd'")
            return P

        # price grid
        Sgrid = dynamics.S0 * np.exp(np.linspace(self.N, -self.N, num=2*self.N+1, endpoint=True)*deltax)

        price_of_put = np.maximum(contract.putstrike - Sgrid, 0.0) #write code to compute this

        call_flag = False

        for step in np.linspace(self.N-1, 0, num=self.N, endpoint=True):
            step = int(step)
            Sgrid_t = dynamics.S0 * np.exp(np.linspace(step, -step, num=2*step + 1, endpoint=True)*deltax)
            t = step * deltat

            if t - deltat < contract.callexpiry:
                if not call_flag:
                    call_flag = True
                    Pgrid = price_of_put
                    price_of_call_on_put = np.maximum(Pgrid - contract.callstrike, 0.0)

            Pu = prob(Sgrid_t, t, 'u')
            Pm = prob(Sgrid_t, t, 'm')
            Pd = prob(Sgrid_t, t, 'd')

            price_of_put = disc * (price_of_put[:-2] * Pu + price_of_put[1:-1] * Pm + price_of_put[2:] * Pd)

            american_put = np.maximum(contract.putstrike - Sgrid_t, 0.0) # value of excercising now

            price_of_put = np.maximum(price_of_put, american_put)

            if call_flag:
                price_of_call_on_put = disc * (price_of_call_on_put[:-2] * Pu +
                                               price_of_call_on_put[1:-1] * Pm +
                                               price_of_call_on_put[2:] * Pd)
    
        return (price_of_put[0], price_of_call_on_put[0])


# Problem 1.

In [5]:
hw2dynamics = localvolDynamics(S0=100, r=0.06, q=0.01, maxvol=0.6,
                               localvol=lambda S, t: np.minimum(0.2 + 5 * np.log(S / 100) ** 2 + 0.1 * np.exp(-t), 0.6))

# Note that hw2dynamics.localvol is a function
# that may be invoked in the usual way, for example:
# hw2dynamics.localvol( exchangerate , time )

In [6]:
hw2contract = CallOnAmericanPut(putexpiry=0.75, putstrike=95, callexpiry=0.25, callstrike=10)

In [7]:
hw2tree = TrinomTree(N = 1500)  #change if necessary to get $0.01 accuracy, in your judgment

In [8]:
(answer_part_a, answer_part_b) = hw2tree.price_compound_localvol(hw2contract,hw2dynamics)

In [9]:
(answer_part_a, answer_part_b)

(7.006141204521073, 1.593833781078742)

# Problem 2.

## (a)

The Black-Scholes call price at time-0 is:

$$
C = S_0 N(d_1) - K e^{-rT} N(d_2)
$$

where

$$
d_1 = \frac{\log\left( \frac{S_0}{K} \right) + rT}{\sigma \sqrt{T}} + \frac{\sigma \sqrt{T}}{2}
$$

and

$$
d_2 = \frac{\log\left( \frac{S_0}{K} \right) + rT}{\sigma \sqrt{T}} - \frac{\sigma \sqrt{T}}{2}
$$

Therefore, when we are considering at-the-money-spot situations ($K = S_0$):

$$
d_{1, 2} = \frac{rT}{\sigma \sqrt{T}} \pm \frac{\sigma \sqrt{T}}{2}
$$

Now, for small $|x|$, the normal cdf function ($N$) can be expanded in first-order Taylor series as:

$$
N(x) = N(0) + N'(0)x + O(x^2) = \frac{1}{2} + \frac{1}{\sqrt{2 \pi}} x + O(x^2)
$$

the delta of the at-the-money-spot call option would be expressed below applying a first-order Taylor expansion:

$$
\frac{\delta C}{\delta S} = N(d_1) \approx \frac{1}{2} + \frac{1}{\sqrt{2 \pi}}
\left(  \frac{rT}{\sigma \sqrt{T}} + \frac{\sigma \sqrt{T}}{2} \right)
$$

In [10]:
def TS_d1_N(x):
    return 0.5 + 1/(np.sqrt(2 * np.pi)) * x

sigma = 0.2
T = 0.25
r = 0.01

atms_d1 = r * T / (sigma * np.sqrt(T)) + sigma * np.sqrt(T)/2

print("Estimated delta of the at-the-money-spot Call:", round(TS_d1_N(atms_d1), 2))

Estimated delta of the at-the-money-spot Call: 0.53


As calculated above, assuming $\sigma = 0.2$, $T = 0.25$, and $r = 0.01$, the delta of the at-the-money-spot Call
can be estimated to two decimal places as $0.53$

## (b)

If we perform a second order Taylor series expansion on $C(S)$ with respect to S,
for small $|S - S_0|$ values:

$$
C(S) = C(S_0) + \frac{\delta C}{\delta S}(S_0) \cdot (S - S_0) +
\frac{1}{2} \frac{\delta^2 C}{\delta S^2}(S_0) \cdot (S - S_0)^2 +
O(S^3)
$$

Here, we know from the option combination price that:

$$
C(S_0) = 5
$$

Moreover, we know from the dollar delta value that:

$$
S_0 \frac{\delta C}{\delta S}(S_0) = 3
$$

$$
\Leftrightarrow
\frac{\delta C}{\delta S}(S_0) = \frac{3}{4}
$$

Finally, we know from the dollar gamma value that:

$$
\frac{S_0^2}{100} \times \frac{\delta^2 C}{\delta S^2}(S_0) = 0.02
$$

$$
\Leftrightarrow
\frac{\delta^2 C}{\delta S^2}(S_0) = \frac{2}{16} = \frac{1}{8}
$$

Using the above three values and the Taylor Series expansion, $C(S)$ for $S$ near $S_0 = 4$ can be expressed as:

$$
C(S) \approx 5 + \frac{3}{4} (S - 4) + \frac{1}{16} (S - 4)^2
$$

The time-0 value of the contract, given an underlying stock price $3.6$, can be approximated as:

$$
C(3.6) \approx 5 + \frac{3}{4} (3.6 - 4) + \frac{1}{16} (3.6 - 4)^2 = 4.71
$$